# AP CSA FRQ Reflection Blog

---

## FRQ 1 — RobotMover

**The Problem:** You're given a `RobotMover` class that stores a string of moves a robot makes. Part A asks you to build that string randomly, and Part B asks you to search it for patterns. The string looks like `"up_down_left_right_"` — every move followed by an underscore as a separator.

---

### Part A: Constructor

**The Problem Specifically:** Given `numMoves`, fill `moveSequence` with that many random directions (up/down/left/right), each followed by `_`. Every direction should have an equal 25% chance of appearing.

```java
public RobotMover(int numMoves) {
    moveSequence = "";
    String[] directions = {"up", "down", "left", "right"};
    
    for (int i = 0; i < numMoves; i++) {
        moveSequence += directions[(int)(Math.random() * 4)] + "_";
    }
}
```

**Breaking it down:**
- `moveSequence = ""` — initialize to empty string first, otherwise you're concatenating onto null which breaks everything
- `String[] directions = {"up", "down", "left", "right"}` — storing all four options in an array means you can pick one with a single index instead of writing four if/else branches
- `(int)(Math.random() * 4)` — `Math.random()` gives a decimal between 0.0 and 0.999..., multiply by 4 to stretch that range to 0.0–3.999..., then cast to int to chop off the decimal, giving you exactly 0, 1, 2, or 3
- The `+ "_"` is appended in the same line as the direction — easy to forget or misplace outside the loop

**Key learnings:**
- `(int)(Math.random() * n)` is the standard formula for random int from 0 to n-1
- Array indexing replaces if/else chains cleanly here
- String concatenation in a loop: always initialize to `""` first

---

### Part B: countOccurrences

**The Problem Specifically:** Count how many times the string `str` appears inside `moveSequence`. The catch — overlapping matches count too. So `"up_up"` inside `"up_up_up_"` appears at index 0 AND index 3, so the answer is 2, not 1.

```java
public int countOccurrences(String str) {
    int count = 0;
    int index = moveSequence.indexOf(str);
    
    while (index != -1) {
        count++;
        index = moveSequence.indexOf(str, index + 1);
    }
    
    return count;
}
```

**Breaking it down:**
- `moveSequence.indexOf(str)` — finds the first occurrence of `str` and returns its starting index, or -1 if not found
- The while loop keeps going as long as `indexOf` doesn't return -1, meaning there are still matches left
- After each match, `index + 1` moves the search forward by just one position — this is what makes overlapping matches possible. If you moved by `str.length()` instead, you'd jump past any match that shares characters with the next one
- `count++` happens every time a match is found, then the search position updates

**The overlapping problem visualized:**

| moveSequence | str | Advance by length | Advance by 1 |
|---|---|---|---|
| `"up_up_up_"` | `"up_up"` | finds 1 | finds 2 |
| `"right_right_right_"` | `"right_right"` | finds 1 | finds 2 |

**Key learnings:**
- Two-argument `indexOf(str, fromIndex)` is essential here — know it cold
- Always advance by 1 after a match when overlaps are possible
- Loop condition `-1` means no more matches — standard pattern for string traversal

---

## FRQ 2 — LapTracker

**The Problem:** Build a class that tracks total laps walked around a track. Every nth call to `addLaps`, the count resets to 0 before adding. The tricky part is the timing — the reset fires at the *start* of the call that comes after every nth call, not at the end of the nth call itself.

```java
public class LapTracker {
    private int resetInterval;
    private int lapCount;
    private int callCount;

    public LapTracker(int reset) {
        resetInterval = reset;
        lapCount = 0;
        callCount = 0;
    }

    public int addLaps(int laps) {
        if (callCount % resetInterval == 0 && callCount != 0) {
            lapCount = 0;
        }
        callCount++;
        lapCount += laps;
        return lapCount;
    }
}
```

**Breaking it down:**
- Three instance variables: `resetInterval` stores the constructor parameter, `lapCount` is the running total, `callCount` tracks how many times `addLaps` has been called
- In `addLaps`, the order of operations is critical: check for reset → increment callCount → add laps → return. If you increment callCount before checking, the modulo math breaks
- `callCount % resetInterval == 0` checks if we're at a reset point — but this is also true when `callCount` is 0 (before any calls), which is why `&& callCount != 0` is necessary to skip the very first call
- After a reset, `lapCount = 0` and then `lapCount += laps` immediately adds the new laps, so the return value is just the new laps alone

**Reset timing traced through the example (reset every 3 calls):**

| Call # | addLaps | callCount before check | Reset fires? | Return |
|---|---|---|---|---|
| 1 | `addLaps(8)` | 0 | No (callCount == 0) | 8 |
| 2 | `addLaps(12)` | 1 | No | 20 |
| 3 | `addLaps(10)` | 2 | No | 30 |
| 4 | `addLaps(11)` | 3 | Yes, 3%3==0, reset to 0 then +11 | 11 |
| 5 | `addLaps(5)` | 4 | No | 16 |

**Key learnings:**
- Read the example table slowly — the reset timing is the whole puzzle
- Three instance variables, no more, no less
- Order inside `addLaps` matters — check before incrementing
- The `&& callCount != 0` guard is easy to miss and will cause a wrong answer on call #1

---

## FRQ 3 — playerWithClosestScore

**The Problem:** The `PlayerAnalysis` class holds an `ArrayList` of `Player` objects. Each player has an ID (String) and a score (int). Given a `targetScore`, find and return the ID of the player whose score is closest to it. Ties can return either player.

```java
public String playerWithClosestScore(int targetScore) {
    String closestID = playerList.get(0).getID();
    int closestDiff = Math.abs(playerList.get(0).getScore() - targetScore);

    for (Player p : playerList) {
        int diff = Math.abs(p.getScore() - targetScore);
        if (diff < closestDiff) {
            closestDiff = diff;
            closestID = p.getID();
        }
    }
    return closestID;
}
```

**Breaking it down:**
- Before the loop, you seed your "best so far" with the first player in the list — both their ID and their difference from the target. This gives the loop something real to compare against from the very first iteration
- `Math.abs(p.getScore() - targetScore)` computes the absolute difference — how far away this player's score is from the target. Without `Math.abs`, a score lower than the target would give a negative number and always look "smaller" than it should
- Inside the loop, if the current player's difference is strictly less than the best seen so far, you update both variables together — you need both the new minimum difference and the ID that goes with it
- After the loop, `closestID` holds the winner

**Traced through the example (targetScore = 1000):**

| Player | Score | Score - 1000 (absolute) | New best? |
|---|---|---|---|
| DJK | 1090 | 90 | Yes (first) |
| CJL | 2800 | 1800 | No |
| JOY | 2000 | 1000 | No |
| BEN | 500 | 500 | No |
| PTT | 3500 | 2500 | No |
| JAY | 4500 | 3500 | No |

**Key learnings:**
- Seed with `playerList.get(0)` — starting with 0 or null as your baseline leads to wrong answers
- `Math.abs()` is non-negotiable when comparing distances
- Always call `getScore()` and `getID()` — the exam expects you to use the interface, not access fields directly
- Update both `closestDiff` and `closestID` together — forgetting one breaks the whole thing

---

## FRQ 4 — countOrderedRows

**The Problem:** You have a 2D array of Strings called `grid`. A row is "ordered" if every string in it has a length greater than or equal to the string before it — shortest to longest, left to right. Count how many rows in the grid meet this condition.

```java
public int countOrderedRows() {
    int count = 0;

    for (int row = 0; row < grid.length; row++) {
        boolean ordered = true;

        for (int col = 1; col < grid[row].length; col++) {
            if (grid[row][col].length() < grid[row][col - 1].length()) {
                ordered = false;
            }
        }

        if (ordered) {
            count++;
        }
    }

    return count;
}
```

**Breaking it down:**
- The outer loop walks through every row using `grid.length` (number of rows)
- `boolean ordered = true` is declared fresh inside the outer loop — it resets for every new row, which is what you want
- The inner loop starts at `col = 1`, not 0 — because you're comparing each element to the one before it (`col - 1`), and there's nothing before index 0
- `grid[row][col].length() < grid[row][col - 1].length()` compares the string lengths, not the strings alphabetically. If the current string is shorter than the previous one, the row is not ordered
- Once `ordered` flips to false, there's no reason to flip it back — any single violation disqualifies the whole row
- After the inner loop finishes, if `ordered` is still true, that row passed and gets counted

**Traced through the example:**

| Row | Words | Lengths | Ordered? |
|---|---|---|---|
| 0 | cat, crane, people, hamster | 3, 5, 6, 7 | Yes — strictly increasing |
| 1 | dog, slate, bat, green | 3, 5, 3, 5 | No — bat(3) < slate(5) |
| 2 | blue, audio, adieu, snazzy | 4, 5, 5, 6 | Yes — equal counts too |
| 3 | yellow, shirt, hat, store | 6, 5, 3, 5 | No — shirt(5) < yellow(6) |
| 4 | about, media, stone, canoe | 5, 5, 5, 5 | Yes — all equal is fine |

**Key learnings:**
- Inner loop starts at `col = 1`, always — starting at 0 causes an out-of-bounds error on `col - 1`
- Compare `.length()` values, not the strings themselves
- Equal lengths still count as ordered — the condition is `<` not `<=`
- `grid.length` = number of rows, `grid[row].length` = number of columns in that row
- The boolean flag pattern is the cleanest way to handle "did anything go wrong in this row"